In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
import streamlit as st
import sklearn

In [3]:
inferencia_path = "../data/raw/inferencia/ventas_2025_inferencia.csv"
inferencia_df = pd.read_csv(inferencia_path)
display(inferencia_df.head())

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [4]:
# --- Transformación completa de inferencia_df para inferencia ---

# 1. Convertir fecha a datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Crear variables temporales y de calendario
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana_num'] = inferencia_df['fecha'].dt.dayofweek
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_de_semana'] = inferencia_df['dia_semana_num'].isin([5, 6])

# Festivos en España
inferencia_df['es_festivo'] = inferencia_df['fecha'].dt.date.isin(
    holidays.country_holidays('ES', years=inferencia_df['año'].unique())
)

# Black Friday: cuarto viernes de noviembre
def es_black_friday(fecha):
    year = fecha.year
    nov = pd.date_range(start=f'{year}-11-01', end=f'{year}-11-30', freq='D')
    fridays = nov[nov.weekday == 4]
    if len(fridays) >= 4:
        bf_date = fridays[3].date()
        return fecha.date() == bf_date
    return False
inferencia_df['es_black_friday'] = inferencia_df['fecha'].apply(es_black_friday)

# Cyber Monday: lunes siguiente al Black Friday
def es_cyber_monday(fecha):
    year = fecha.year
    nov = pd.date_range(start=f'{year}-11-01', end=f'{year}-11-30', freq='D')
    fridays = nov[nov.weekday == 4]
    if len(fridays) >= 4:
        bf_date = fridays[3]
        cm_date = bf_date + pd.Timedelta(days=3)
        return fecha.date() == cm_date.date()
    return False
inferencia_df['es_cyber_monday'] = inferencia_df['fecha'].apply(es_cyber_monday)

# Primer y último día de mes
inferencia_df['es_primer_dia_mes'] = inferencia_df['dia_mes'] == 1
inferencia_df['es_ultimo_dia_mes'] = inferencia_df['fecha'] == inferencia_df['fecha'] + pd.offsets.MonthEnd(0)
inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week

# 3. Crear variables de lags y media móvil (por producto y año)
lags = [1,2,3,4,5,6,7]
for lag in lags:
    inferencia_df[f'unidades_vendidas_lag{lag}'] = inferencia_df.groupby(['producto_id', 'año'])['unidades_vendidas'].shift(lag)
inferencia_df['unidades_vendidas_mm7'] = inferencia_df.groupby(['producto_id', 'año'])['unidades_vendidas'].rolling(window=7).mean().reset_index(level=[0,1], drop=True)

# 4. Crear variable descuento_porcentaje
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# 5. Añadir precios de competencia y ratio_precio
competencia_df = pd.read_csv('../data/raw/entrenamiento/competencia.csv')
competencia_df['fecha'] = pd.to_datetime(competencia_df['fecha'])
inferencia_df = inferencia_df.merge(
    competencia_df,
    on=['fecha', 'producto_id'],
    how='left'
)
inferencia_df['precio_competencia'] = inferencia_df[['Amazon', 'Decathlon', 'Deporvillage']].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# 6. One hot encoding de nombre, categoria y subcategoria
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']
inferencia_df = pd.get_dummies(inferencia_df, columns=['nombre_h', 'categoria_h', 'subcategoria_h'])

# 7. Eliminar registros de octubre y dejar solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

# 8. Guardar el DataFrame transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print('DataFrame de inferencia transformado y guardado en data/processed/inferencia_df_transformado.csv')

KeyError: "None of [Index(['Amazon', 'Decathlon', 'Deporvillage'], dtype='object')] are in the [columns]"

In [ ]:
# --- Comprobación de columnas tras el merge y corrección ---
print('Columnas tras el merge:', inferencia_df.columns.tolist())
competencia_cols = ['Amazon', 'Decathlon', 'Deporvillage']
missing_cols = [col for col in competencia_cols if col not in inferencia_df.columns]
if missing_cols:
    print(f'Las siguientes columnas de competencia no están en inferencia_df: {missing_cols}')
    inferencia_df['precio_competencia'] = np.nan
    inferencia_df['ratio_precio'] = np.nan
else:
    inferencia_df['precio_competencia'] = inferencia_df[competencia_cols].mean(axis=1)
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']


In [14]:
inferencia_df

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,unidades_vendidas_lag5,unidades_vendidas_lag6,unidades_vendidas_lag7,unidades_vendidas_mm7,descuento_porcentaje,Amazon_y,Decathlon_y,Deporvillage_y,precio_competencia,ratio_precio
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,...,NaN,NaN,NaN,NaN,-1.626087,NaN,NaN,NaN,NaN,NaN
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,...,NaN,NaN,NaN,NaN,5.103704,NaN,NaN,NaN,NaN,NaN
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,...,NaN,NaN,NaN,NaN,0.929412,NaN,NaN,NaN,NaN,NaN
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,...,NaN,NaN,NaN,NaN,1.586667,NaN,NaN,NaN,NaN,NaN
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,...,NaN,NaN,NaN,NaN,1.371429,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,2025-11-30,PROD_020,Quechua MH500,Outdoor,Ropa Montaña,80,False,NaN,79.64,NaN,...,NaN,NaN,NaN,NaN,-0.450000,NaN,NaN,NaN,NaN,NaN
884,2025-11-30,PROD_021,Manduka PRO Yoga Mat,Wellness,Esterilla Yoga,130,True,NaN,130.00,NaN,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
885,2025-11-30,PROD_022,Gaiam Premium Yoga Block,Wellness,Bloque Yoga,20,False,NaN,20.18,NaN,...,NaN,NaN,NaN,NaN,0.900000,NaN,NaN,NaN,NaN,NaN
886,2025-11-30,PROD_023,Liforme Yoga Pad,Wellness,Rodillera Yoga,35,False,NaN,34.79,NaN,...,NaN,NaN,NaN,NaN,-0.600000,NaN,NaN,NaN,NaN,NaN


In [15]:
# --- Filtrar solo registros de noviembre y guardar el resultado ---
inferencia_df_noviembre = inferencia_df[inferencia_df['mes'] == 11].copy()
inferencia_df_noviembre.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print('DataFrame de inferencia de noviembre guardado en data/processed/inferencia_df_transformado.csv')

DataFrame de inferencia de noviembre guardado en data/processed/inferencia_df_transformado.csv


In [17]:
inferencia_df_noviembre.columns

Index(['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria',
       'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta',
       'ingresos', 'Amazon_x', 'Decathlon_x', 'Deporvillage_x', 'año', 'mes',
       'dia_mes', 'dia_semana_num', 'dia_semana', 'es_fin_de_semana',
       'es_festivo', 'es_black_friday', 'es_cyber_monday', 'es_primer_dia_mes',
       'es_ultimo_dia_mes', 'semana_año', 'unidades_vendidas_lag1',
       'unidades_vendidas_lag2', 'unidades_vendidas_lag3',
       'unidades_vendidas_lag4', 'unidades_vendidas_lag5',
       'unidades_vendidas_lag6', 'unidades_vendidas_lag7',
       'unidades_vendidas_mm7', 'descuento_porcentaje', 'Amazon_y',
       'Decathlon_y', 'Deporvillage_y', 'precio_competencia', 'ratio_precio'],
      dtype='object')